# Get a LSST Simulated Cadence for Arbitrary Sky Position
___


___

In [ ]:
"""

"""
# %matplotlib inline
%matplotlib notebook


from __future__ import print_function

try:
    from importlib import reload
except:
    pass


from matplotlib import pyplot as plt
import matplotlib.colors as mpl_colors 
from matplotlib import colorbar as mpl_colorbar
from matplotlib import rc
rc('text', usetex=True)

import os
import warnings

import numpy as np
import pandas as pd
import astropy as ap
import sfdmap

import sqlite3
from sqlalchemy import create_engine

from astropy import units as u
from astropy.coordinates import SkyCoord

import lsst_tools.utils as utils 
import lsst_tools.utils.colours as colours 

from multiprocessing import Pool, cpu_count
from contextlib import closing

Connect to .db file that contains the opsim output, read into a dataframe, `opsimdf`

In [ ]:
# %%timeit ## 1 loop, best of 3: 1min 31s per loop 
# opsimdbpath = os.environ.get('OPSIMDBPATH')
# print(opsimdbpath)
opsimdbpath = "/Users/berto/data/LSST/OpSimOutputDBs/astro_lsst_01_1068_sqlite.db"

conn = create_engine('sqlite:///'+opsimdbpath, echo = False)
opsimdf = pd.read_sql_table('Summary', con=conn)

In [ ]:
opsimdf

In [ ]:
len(opsimdf["fieldID"].unique())

In [ ]:
#%%timeit ## The slowest run took 39.04 times longer than the fastest. This could mean that an intermediate result is being cached.
#         ## 1000 loops, best of 3: 246 µs per loop
n = 1
pos_df = utils.generate_coordinates(n)

Add extinction to the dataframe

In [ ]:
#%%timeit ## 1 loop, best of 3: 88.6 ms per loop
df = utils.find_MW_extinction(pos_df) 

Check the positions are sensible

In [ ]:
#%%timeit ## 1 loop, best of 3: 235 ms per loop
utils.plot_position_points(df)

Connecting to `.db` takes ages (~1min), and is a pain if you mess up, so create a 'working' instance to fiddle with

In [ ]:
working_df = opsimdf

## Identifying Fields
___
We want to find out which field(s) the position is in. Create table that tells us the field centres, and find the edges.

In [ ]:
# working_df = utils.get_field_corners(working_df.drop_duplicates("fieldID"))[['fieldRA', 'fieldDec']]
working_df = utils.get_field_corners(pd.DataFrame(working_df.drop_duplicates("fieldID"))[['fieldID', 'fieldRA', 'fieldDec']])

In [ ]:
working_df

In [ ]:
field_df = working_df[(working_df["RA_upper"] >= df["theta"][0] + np.pi) &
                (working_df["RA_lower"] <= df["theta"][0] + np.pi) &
                (working_df["Dec_lower"] >= df["phi"][0])  &
                (working_df["Dec_upper"] <= df["phi"][0])]

In [ ]:
reload(utils)
utils.plot_field(working_df)

In [ ]:
df = opsimdf

In [ ]:
## Work out how often each field is visited
min_visits = df["fieldID"].value_counts().min()
max_visits = df["fieldID"].value_counts().max()

print(min_visits, ",",max_visits)

df['count'] = df.groupby('fieldID').cumcount()

In [ ]:
ddf = [1427,744,2412,290,2786  ]

In [ ]:
cmap = plt.get_cmap("viridis")
# df["cmap_visit_value"] = (df["count"] - min_visits )/(max_visits - min_visits)
df["cmap_visit_value"] = np.log((df["count"] + 1)/(max_visits+1))

In [ ]:
df["cmap_visit_value"] = df["cmap_visit_value"] + np.abs(df["cmap_visit_value"].min())
df["cmap_visit_value"] = df["cmap_visit_value"]/df["cmap_visit_value"].max()

In [ ]:
df["cmap_visit_value"].hist(bins = 200)

In [ ]:
df["fieldID"].value_counts().hist(bins = 200, range = [0, 2000])

In [ ]:
df = utils.get_field_corners(df)

In [ ]:
night_group = df.groupby("night")

In [ ]:
#TEST
# i = 0
# group = night_group.groups[0]
# outpath = "/Users/berto/plots/LSST/cadence/minion/skymap_night_"
# fig = plt.figure()
# fig.subplots_adjust(left = 0.09, bottom = 0.13, top = 0.99,
#                     right = 0.97, hspace=0, wspace = .1)

# ax_aitoff = fig.add_subplot(111, projection="aitoff")
# ax_aitoff.grid(True)
# ax_aitoff.scatter(df["fieldRA"][group] - np.pi, df["fieldDec"][group], color = cmap(df["cmap_visit_value"][group]))
# fig.savefig(outpath + str(i)+".pdf", format = 'pdf', dpi=500)
# plt.close()

In [ ]:
# for group in night_group.groups:
#     print(night_group.groups[0])

In [ ]:
df_to_plot = df[["fieldID", "fieldRA", "fieldDec", "cmap_visit_value"]]
df_to_plot["zorder"] = df_to_plot["fieldID"]*0
df_to_plot["zorder"].loc[df_to_plot["fieldID"].isin(ddf)] = 99

In [ ]:
df_to_plot.loc[night_group.groups[0]]

In [ ]:
x = len(night_group.groups)

In [ ]:
str('%05d' % x)

In [ ]:
df_to_plot

In [ ]:
# def plot_obs(df):
#     outpath = "/Users/berto/plots/LSST/cadence/minion/Test_skymap_night_"

#     df_to_plot = df[["fieldID", "fieldRA", "fieldDec", "cmap_visit_value"]]
#     df_to_plot["zorder"] = df_to_plot["fieldID"]*0
#     df_to_plot["zorder"].loc[df_to_plot["fieldID"].isin(ddf)] = 99

#     norm = mpl_colors.LogNorm(vmin=max_visits, vmax=min_visits)


#     for i, group in enumerate(np.arange(10)):
#     # for i, group in enumerate(night_group.groups):

#         if i == 0:
#             fig = plt.figure()
#             fig.subplots_adjust(left = 0.09, bottom = 0.13, top = 0.99,
#                                 right = 0.97, hspace=0, wspace = .1)

#     #         ax_aitoff = fig.add_subplot(111, projection="aitoff")      
#             ax_aitoff = plt.subplot2grid((1,24), (0,0), colspan = 22, rowspan = 1, projection="aitoff")
#             ax_aitoff.grid(True)
#             ax_cbar = plt.subplot2grid((1,24), (0,22), colspan = 1, rowspan = 1)
#     #         ax_active = fig.add_subplot(111, projection="aitoff")


#             working_df_to_plot = df_to_plot.loc[night_group.groups[group]]
#         else:
#             working_df_to_plot = pd.concat([working_df_to_plot,df_to_plot.loc[night_group.groups[group]]])
#             working_df_to_plot.drop_duplicates(subset=['fieldID'], inplace=True, keep="last")

#     #     ax_aitoff.scatter(working_df_to_plot["fieldRA"][night_group.groups[group]] - np.pi, 
#     #               working_df_to_plot["fieldDec"][night_group.groups[group]], 
#     #               color = cmap(working_df_to_plot["cmap_visit_value"][night_group.groups[group]]))

#         ax_aitoff.scatter(working_df_to_plot["fieldRA"] - np.pi, 
#                           working_df_to_plot["fieldDec"], 
#                           color = cmap(working_df_to_plot["cmap_visit_value"]))

#         ax_aitoff.scatter(working_df_to_plot.loc[df_to_plot["zorder"] == 99,["fieldRA"]] - np.pi, 
#                           working_df_to_plot.loc[df_to_plot["zorder"] == 99,["fieldDec"]], 
#                           color = cmap(working_df_to_plot[working_df_to_plot["zorder"] == 99]["cmap_visit_value"])
#                           )

#     #     fig_cont = fig

#     #     ax_active.scatter(working_df_to_plot["fieldRA"] - np.pi, 
#     #                       working_df_to_plot["fieldDec"], 
#     #                       color = cmap(working_df_to_plot["cmap_visit_value"]),
#     #                       linewidths = 0.5 , edgecolors = colours.hex["black"])

#         cb1 = mpl_colorbar.ColorbarBase(ax_cbar, cmap=cmap,
#                                     norm=norm)
#         fig.savefig(outpath + str('%05d' % i)+".png", format = 'png', dpi=200)

#     #     ax_active.clear()
#     #     print(len(working_df_to_plot))
#     # plt.close()

In [ ]:
df_to_plot = df[["fieldID", "fieldRA", "fieldDec", "cmap_visit_value"]]

# outpath = "/Users/berto/plots/LSST/cadence/minion/Test_skymap_night_"
outpath = os.path.join(os.environ["HOME"], "plots/LSST/cadence/astro_lsst_01_1068/skymap_night_")

df_to_plot["zorder"] = df_to_plot["fieldID"]*0
df_to_plot["zorder"].loc[df_to_plot["fieldID"].isin(ddf)] = 99
norm = mpl_colors.LogNorm(vmin=max_visits, vmax=min_visits)


# for i, group in enumerate(np.arange(4)):
for i, group in enumerate(night_group.groups):

    if i == 0:
        fig = plt.figure()
        fig.subplots_adjust(left = 0.05, bottom = 0.05, top = 0.99,
                            right = 0.97, hspace=0, wspace = .1)
    
#         ax_aitoff = plt.subplot2grid((1,24), (0,0), colspan = 22, rowspan = 1, projection="aitoff")
#         ax_aitoff.grid(True)
#         ax_active = plt.subplot2grid((1,24), (0,0), colspan = 22, rowspan = 1, projection="aitoff")
#         ax_active.grid(True)
    
        ax_aitoff = fig.add_subplot(111, projection = "aitoff")
        ax_active = fig.add_subplot(111, projection = "aitoff")
        ax_aitoff.grid(True)
        ax_active.grid(True)

#         ax_cbar = plt.subplot2grid((1,24), (0,22), colspan = 1, rowspan = 1)
        ax_cbar = fig.add_subplot(20,1,19)
        cb1 = mpl_colorbar.ColorbarBase(ax_cbar, cmap=cmap,
                                norm=norm, orientation='horizontal')
        cb1.set_label(r'$\textnormal{Number of Visits}$')
        
        working_df_to_plot = df_to_plot.loc[night_group.groups[group]]

    else:
#         ax_active = fig.add_subplot(111, projection = "aitoff")
        working_df_to_plot = pd.concat([working_df_to_plot,df_to_plot.loc[night_group.groups[group]]])
    
        working_df_to_plot.drop_duplicates(subset=['fieldID'], inplace=True, keep="last")

    s1 = ax_aitoff.scatter(working_df_to_plot["fieldRA"] - np.pi, 
                      working_df_to_plot["fieldDec"], 
                      color = cmap(working_df_to_plot["cmap_visit_value"]))

    
    s2 = ax_aitoff.scatter(working_df_to_plot.loc[df_to_plot["zorder"] == 99,["fieldRA"]] - np.pi, 
                      working_df_to_plot.loc[df_to_plot["zorder"] == 99,["fieldDec"]], 
                      color = cmap(working_df_to_plot[working_df_to_plot["zorder"] == 99]["cmap_visit_value"])
                      )
    
    s_a = ax_active.scatter(df_to_plot.loc[night_group.groups[group]]["fieldRA"] - np.pi, 
                      df_to_plot.loc[night_group.groups[group]]["fieldDec"], 
                      color = cmap(df_to_plot.loc[night_group.groups[group]]["cmap_visit_value"]),
                      edgecolor = colours.hex["black"])

    ## Works
#     s1 = ax_aitoff.scatter(x, data[str(group)], color = colours.hex["pomegranite"])
#     s_a = ax_active.scatter(x, data[str(group)], color = colours.hex["pomegranite"], edgecolor = "black")
    
    ## Works
#     s1 = ax_aitoff.scatter(working_df_to_plot["fieldRA"] - np.pi, 
#                            working_df_to_plot["fieldDec"], 
#                       color = colours.hex["pomegranite"])

#     s_a = ax_active.scatter(df_to_plot.loc[night_group.groups[group]]["fieldRA"] - np.pi, 
#                       df_to_plot.loc[night_group.groups[group]]["fieldDec"], 
#                       color = colours.hex["pomegranite"],
#                       edgecolor = colours.hex["black"])
#     print(len(working_df_to_plot["fieldDec"]))
    plt.draw()
    fig.savefig(outpath + str('%05d' % i)+".png", format = 'png', dpi=200)
    s_a.remove()
#     del(s_a)
    
#     fig.delaxes(ax_active)


In [ ]:
ones = np.ones(10)

In [ ]:
len(ones[0:])

In [ ]:
working_df_to_plot[working_df_to_plot["zorder"] == 99]["cmap_visit_value"]

In [ ]:
df_to_plot.loc[df_to_plot["zorder"] == 99,["fieldRA"]]

In [ ]:
# working_df_to_plot["fieldID"] in df_to_plot["fieldID"]
# df1 = working_df_to_plot.head(10)
# df2 = working_df_to_plot.tail(10)
df1 = df_to_plot.loc[df_to_plot["fieldID"] == 505].head(1)
df2 = df_to_plot.loc[df_to_plot["fieldID"] == 505].tail(1)

In [ ]:
df_to_plot["zorder"].loc[df_to_plot["fieldID"].isin(ddf)] = 1

In [ ]:
df_to_plot["zorder"].loc[df_to_plot["fieldID"].isin(ddf)]

In [ ]:
df2

In [ ]:
df1

In [ ]:
df3 = pd.concat([df1,df2])

In [ ]:
df3

In [ ]:
df3.drop_duplicates(subset=['fieldID'], inplace=True, keep="last")

In [ ]:
df3

In [ ]:
df.loc[df["fieldID"] == 505].head(10)

In [ ]:
# outpath = "/Users/berto/plots/LSST/cadence/minion/skymap_"


# for row in df.iterrows():
#     fig = plt.figure()
#     fig.subplots_adjust(left = 0.09, bottom = 0.13, top = 0.99,
#                         right = 0.97, hspace=0, wspace = .1)

#     ax_aitoff = fig.add_subplot(111, projection="aitoff")
#     ax_aitoff.grid(True)
#     ax_aitoff.scatter(row["fieldRA"] - np.pi, row["fieldDec"], color = cmap(row[cmap_visit_value]))
    
#     fig.savefig(outpath + row["obsHistID"]+".pdf", format = 'pdf', dpi=500)

In [ ]:
data = {}
x = np.arange(10)

for i in np.arange(5):
    data[str(i)] = np.random.random(10)

outpath = "/Users/berto/plots/LSST/cadence/test_"

for i, j in enumerate(data):
    if i == 0:
        fig = plt.figure()
        fig.subplots_adjust(left = 0.05, bottom = 0.01, top = 0.99,
                            right = 0.97, hspace=0, wspace = .1)

        ax = fig.add_subplot(111, projection = "aitoff")
        ax_active = fig.add_subplot(111, projection = "aitoff")
        
    else:
        pass
    
    s1 = ax.scatter(x, data[j], color = colours.hex["pomegranite"])
    s2 = ax_active.scatter(x, data[j], color = colours.hex["pomegranite"], edgecolor = "black")
    
    fig.savefig(outpath + str('%05d' % i)+".png", format = 'png', dpi=200)
    s2.remove()
